# Automatic Differentiation with *autodiff*

## Introduction

The *autodiff* package implements [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation), a technique for computing derivatives of functions that is distinct from both symbolic and numerical differentiation. 

The merit of automatic differentiation is its ability to handle complicated functions without sacrificing the accuracy of the computed derivative. Whereas symbolic differentiation guarantees accuracy, it can be intractable for complex functions. By contrast, numerical differentiation affords ease in implementation and is amenable to any function, but it comes at the expense of accuracy. Automatic, or algorithmic, differentation is an alternative approach which allows derivatives to be computed by a computer program up to machine precision.

Automatic differentiation in science and engineering has been applied to address problems in optimization, root-finding, and implicit time-integration.

## Background

The underlying principle behind automatic differentiation is the chain rule. Any complex function whose derivative we would like to compute can be expressed as the composition of elementary functions and operations. By obeying properties of the chain rule in tracing how the elementary operations make up the final function, an accompanying derivative of the function can be computed.

There are two flavors of algorithm for implementing automatic differentiation: the forward mode and the reverse mode. Here we will focus on the forward mode, which means that derivatives are computed starting from the most primitive building block - the independent variables of the function - to the most complex: the function itself. Following is a simple example of how the forward mode may be used determine the derivative of


$$
\begin{align}
    f(x,y) = \log \left[ 1 + \exp \left( x^2 - y^2 \right) \right]
\end{align}
$$

We can visualize the process by constructing a computational graph. The first two nodes of our graph, $x_1$ and $x_2$, take as input the independent variables of the function $f$. Their gradients $\left(\dfrac{\partial x_i}{\partial x},\dfrac{\partial x_i}{\partial y}\right)$ have the seed values $(1,0)$ and $(0,1)$, respectively.
![](figs/cg1.png)
Next, $x_3$ and $x_4$ are exponentiated to obtain $x_1$ and $x_2$, and their corresponding gradients are defined in terms of $x_1$ and $x_2$, computed in the previous step.
![](figs/cg2.png)
$x_5$ is composed by a subtraction operation between $x_3$ and $x_4$, and followed by raising $e$ to $x_5$ to obtain $x_6$.
![](figs/cg3.png)
Following through the remaining operations, we arrive at $x_8$, which corresponds to the final function $f$:
![](figs/cg4.png)

This procedure also extends naturally to vector-valued functions, for which the computational graph would end with multiple output nodes.

## Software Organization

The following is a proposed directory structure and relevant modules we plan to implement in our package:

```
README.md
docs\
    milestone1.ipynb
    examples\
        intro_AD.py
        multivariate_AD.py
        vector_valued_AD.py
        special_functions.py
autodiff\
    autodiff\
        __init__.py
        autodiff.py
        math.py
        tests\
            __init__.py
            test_autodiff.py
    setup.py
    LICENSE
    README.md
    .travis.yml
    setup.cfg
```

The key modules in this package are:
1. **autodiff\autodiff.py**: defines the core structure of automatic differentiation, including   attributes for accessing values and gradients, and methods for operator overloading. This is the main module users will import.

2. **autodiff\math.py**: defines elementary functions (sin, exp) to be imported by the user and used when defining a function.

4. **docs\examples\intro_AD.py**: An introductory example of using the AutoDiff package, covering univariate, scalar valued functions.
5. **examples\multivariate_AD.py**: A demonstration of automatic differentiation for multivariate functions, as well as retrieving the attributes of vector-valued functions.
6. **examples\special_functions.py**: How to import and use functions such as `sin` and `exp` when defining functions.
7. **autodiff\tests\test_autodiff.py**: A module of test functions that we will use in validating our implementation.

The test suite will be in the tests directory of our package. We envision maintaining our test suite and package on GitHub, with Travis CI and Coveralls for continuous integration. We plan to look into PyPI for our package distribution.

## How to use the *autodiff* package

The following three examples illustrate some common uses of the `autodiff` package.
#### Univariate, scalar-valued function
After importing `autodiff`, a `Var` object is created which represents an independent variable. A function f defined in terms of this object will be differentiated, and its derivative with respect to x accessed through the method `der(x)`.

```Python
from autodiff import autodiff

x = autodiff.Var()

f = 5*x**2

x.set_value(5)
print(f.der(x)) # prints the derivative of f with respect to x
```

#### Multivariate, scalar-valued function
A multivariable function is defined by initializing more than one `Var` object. Then, the derivative with respect to each independent variable may be accessed by passing in the appropriate variable in the argument of the `der()` method. The `grad()` method allows several derivatives to be returned at once. Here we also illustrate the use of special functions `sin` and `cos`, imported from the `autodiff` math module.

```Python
from autodiff import autodiff
from autodiff import math as admath

x = autodiff.Var()
y = autodiff.Var()

f = admath.sin(x)**2 + 2*admath.cos(y)

x.set_value(5)
y.set_value(3)
print(f.der(x)) # prints the derivative of f with respect to x
print(f.der(y)) # prints the derivative of f with respect to y
assert f.grad([x,y]) == [f.der(x), f.der(y)]
```

#### Multivariate, vector-valued function
Vector-valued functions can be defined using the `Array` class of `autodiff`. Then, the `der.()` method returns a list of derivatives whose length is equal to the number of components of f, while the `grad()` method can return a list of lists - the complete Jacobian of the function. 

```Python
from autodiff import autodiff

x = autodiff.Var()
y = autodiff.Var()

f = autodiff.Array([5*x**2+3*y,
                    3*x+2*y**2])

x.set_value(5)
y.set_value(3)
assert f.der(x) == [f[0].der(x),
                    f[1].der(x)]
                    
assert f.grad([x,y]) == [[f[0].der(x), f[0].der(y)],
                         [f[1].der(x), f[1].der(y)]]
```

## Implementation

Core data structures include:

- A list for unique IDs that identify operations

#### The `Operation` Class

The `Operation` Class forms the core data structure for automatic differentiation. Subclasses of the `Operation` include `Var`, `Constant`, operators, and elementary functions. Its attributes and methods are the following:

```
Class Operation:
      Attributes: 
          ID
          _value
      Methods:
          __eq__(self, other)
          __add__(self, other)
          __radd__(self, other)
          __sub__ (self, other)
          __rsub__ (self, other)
          __mul__(self, other)
          __rmul__(self, other)
          __div__ (self, other)
          __rdiv__ (self, other)
          __neg__ (self)
          __pow__(self, power, modulo=None)
          __rpow__(self, other)
          evaluate(self)
          der(self, op)
```
    
#### The `Var` Subclass
Instances of the `Var` subclass represent independent variables. User-defined functions that can be differentiated by our package are expressed in terms of `Var` instances. The `Var` Class inherits from its `Operation` superclass, and defines an additional method:

```
Class Var(Operation):
      Methods:
          set_value(self, value)
```

#### The `Constant` Subclass
The `Constant` Subclass allows our package to handle constants in a user-defined function.

#### Elementary operations
Elementary operations include `Addition`, `Multiplication`, `Subtraction`, `Division`, `Negation`, `Power`, and overload the existing operations. They are defined as subclasses of `Operation`. User-defined functions involving these operations will be instances of an operation subclass, corresponding to the operation applied at the last step in the computational graph.

#### Elementary functions
Elementary functions include `sin`, `cos`, `tan`, and their inverses,`exp` and `log`, and `abs`. A user wishing to define a function with these operations must import them from the `autodiff.math` module prior to defining the function. As with elementary operations, user-defined functions using elementary functions will be instances of the elementary function if it is the last step applied in the computational graph. An external dependency of our package is `numpy`, which allows us to carry out the elementary operations within our subclasses.

#### The `Array` Class
The `Array` Class is used to define vector-valued functions. A user wishing to define a vector valued function can express their function as an instance of `Array`, which takes as its argument a list of the components of the function. Its methods are the following:

```
Array:
      Methods:
          der(self, op)
```

#### The `IDAllocator` Class
The `IDAllocator` Class is a helper class that allocates a unique ID to operations and variables. Its attributes and methods are the following:

```
IDAllocator:
      Attributes:
          ids
      Methods:
          allocate_id(cls)
```